## Data Exploration  
The goal of this notebook is to get to know the data that will be used in this project.  

**Tasks:**  
1. Load Data
2. Explore/Clean Features

**1. Load Data**

In [3]:
import nflreadpy as nfl
import pandas as pd
import numpy as np

In [4]:
#only load a couple seasons to keep exploration fast
pbp = nfl.load_pbp([2022, 2024])

In [5]:
df = pbp.to_pandas()

In [6]:
df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1.0,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,None,None,None,...,0.0,1.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43.0,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0.0,1.0,-0.443521,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68.0,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0.0,1.0,1.468819,NaN,NaN,NaN,NaN,NaN,0.440373,-44.037291
3,89.0,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0.0,1.0,-0.492192,0.727261,6.988125,6.0,0.60693,0.227598,0.389904,61.009598
4,115.0,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,0.0,1.0,-0.325931,NaN,NaN,NaN,NaN,NaN,0.443575,-44.357494


In [7]:
df.shape

(98926, 372)

**2. Data Exploration**

The idea behind this project is to build a model that acts as an offensive coordinator. Therefore, the model should only have access to data that an NFL OC would know in the moment. Therefore, we factor out unnecessary columns based on that. We also get rid of entries outside the scope of the model (special teams is an example). 

In [10]:
df = df[
(df['play_type'].isin(['run', 'pass'])) & (df['special'] == 0) & (df['two_point_attempt'] == 0)
& (df['sack'] == 0) & (df['qb_scramble'] == 0)
]

In [11]:
df.shape

(65367, 372)

In [12]:
#building out our variable to be predicted by the model. Should be a play subtype ex: 'short pass left'
def classify_play(row):
    if row.play_type == 'run':
        if row.run_location == 'middle':
            return 'inside run'
        elif row.run_location in ['left', 'right']:
            return 'outside run'
    elif row.play_type == 'pass':
        if pd.notna(row.pass_length) and pd.notna(row.pass_location):
            return f"{row.pass_length} pass {row.pass_location}"
    return 'other'

df['play_subtype'] = df.apply(classify_play, axis=1)


In [13]:
df['play_subtype'].value_counts(dropna = False)

play_subtype
outside run          20680
short pass right     12600
short pass left      11286
inside run            6989
short pass middle     6836
deep pass right       2731
deep pass left        2634
deep pass middle      1359
other                  252
Name: count, dtype: int64

In [14]:
df = df[df['play_subtype'] != 'other']

In [15]:
df['play_subtype'].value_counts(dropna = False)

play_subtype
outside run          20680
short pass right     12600
short pass left      11286
inside run            6989
short pass middle     6836
deep pass right       2731
deep pass left        2634
deep pass middle      1359
Name: count, dtype: int64

In [16]:
df['play_subtype'].count()

65115

In [17]:
df.shape

(65115, 373)

In [18]:
#Engineering Defensive Historical Features to help prediction in a realistic fashion

In [19]:
#Create new DataFrame that contains weekly defensive statistics for each team and season

In [20]:
defsummary = (
    df.groupby(['defteam', 'season', 'week']).agg({
        'yards_gained': 'mean',
        'play_type': lambda x: (x == 'pass').mean(),
        'rush': (lambda x: df.loc[x.index, 'yards_gained'][df.loc[x.index, 'play_type'] == 'run'].mean())
    })
    .rename(columns = {'yards_gained': 'avg_yds_allowed_per_play',
                       'play_type': 'def_pass_rate',
                      'rush': 'avg_rush_yds_allowed_play'})
    .reset_index()
)

In [21]:
defsummary = defsummary.sort_values(['defteam','season', 'week'])

In [22]:
defsummary['def_pass_rate_last3'] = (
    defsummary.groupby('defteam')['def_pass_rate']
        .shift(1)
        .rolling(3, min_periods = 1)
        .mean()
        .reset_index(level = 0, drop = True)
    )

In [23]:
defsummary['avg_yds_pp_alwd_last3'] = (
    defsummary.groupby('defteam')['avg_yds_allowed_per_play']
        .shift(1)
        .rolling(3, min_periods = 1)
        .mean()
        .reset_index(level = 0, drop = True)
    )

In [24]:
defsummary['avg_rush_pp_alwd_last3'] = (
    defsummary.groupby('defteam')['avg_rush_yds_allowed_play']
        .shift(1)
        .rolling(3, min_periods = 1)
        .mean()
        .reset_index(level = 0, drop = True)
    )

In [25]:
defsummary.head()

,defteam,season,week,avg_yds_allowed_per_play,def_pass_rate,avg_rush_yds_allowed_play,def_pass_rate_last3,avg_yds_pp_alwd_last3,avg_rush_pp_alwd_last3
0,ARI,2022,1,7.790323,0.629032,5.347826,NaN,NaN,NaN
1,ARI,2022,2,5.474576,0.661017,3.550000,0.629032,7.790323,5.347826
2,ARI,2022,3,8.261905,0.595238,5.764706,0.645025,6.632449,4.448913
3,ARI,2022,4,4.937500,0.750000,3.333333,0.628429,7.175601,4.887511
4,ARI,2022,5,5.287879,0.545455,3.666667,0.668752,6.224660,4.216013


In [26]:
df = df.merge(defsummary[['defteam', 'season', 'week', 'def_pass_rate_last3', 'avg_yds_pp_alwd_last3', 'avg_rush_pp_alwd_last3']],
              on = ['defteam', 'season', 'week'],
              how = 'left')

In [27]:
df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe,play_subtype,def_pass_rate_last3,avg_yds_pp_alwd_last3,avg_rush_pp_alwd_last3
0,68.0,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,NaN,NaN,NaN,NaN,0.440373,-44.037291,outside run,0.656250,5.358173,4.320055
1,89.0,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,6.988125,6.0,0.606930,0.227598,0.389904,61.009598,short pass left,0.656250,5.358173,4.320055
2,115.0,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,NaN,NaN,NaN,NaN,0.443575,-44.357494,outside run,0.656250,5.358173,4.320055
3,136.0,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,NYJ,home,BAL,...,NaN,NaN,NaN,NaN,0.963242,3.675753,short pass right,0.656250,5.358173,4.320055
4,202.0,2022_01_BAL_NYJ,2022091107,NYJ,BAL,REG,1,BAL,away,NYJ,...,10.545964,9.0,0.606959,0.241949,0.479318,52.068213,short pass right,0.488917,5.883399,3.732620


In [28]:
df = df[['season', 'week', 'game_id', 'posteam', 'defteam', 'qtr', 'down', 'ydstogo', 'yardline_100',
         'score_differential', 'def_pass_rate_last3', 'avg_yds_pp_alwd_last3', 'avg_rush_pp_alwd_last3',
         'play_subtype']]

In [29]:
df.head(20)

,season,week,game_id,posteam,defteam,qtr,down,ydstogo,yardline_100,score_differential,def_pass_rate_last3,avg_yds_pp_alwd_last3,avg_rush_pp_alwd_last3,play_subtype
0,2022,1,2022_01_BAL_NYJ,NYJ,BAL,1.0,1.0,10.0,78.0,0.0,0.656250,5.358173,4.320055,outside run
1,2022,1,2022_01_BAL_NYJ,NYJ,BAL,1.0,1.0,10.0,59.0,0.0,0.656250,5.358173,4.320055,short pass left
2,2022,1,2022_01_BAL_NYJ,NYJ,BAL,1.0,2.0,10.0,59.0,0.0,0.656250,5.358173,4.320055,outside run
3,2022,1,2022_01_BAL_NYJ,NYJ,BAL,1.0,3.0,5.0,54.0,0.0,0.656250,5.358173,4.320055,short pass right
4,2022,1,2022_01_BAL_NYJ,BAL,NYJ,1.0,1.0,10.0,72.0,0.0,0.488917,5.883399,3.732620,short pass right
5,2022,1,2022_01_BAL_NYJ,BAL,NYJ,1.0,2.0,6.0,68.0,0.0,0.488917,5.883399,3.732620,short pass left
6,2022,1,2022_01_BAL_NYJ,BAL,NYJ,1.0,3.0,2.0,64.0,0.0,0.488917,5.883399,3.732620,outside run
7,2022,1,2022_01_BAL_NYJ,BAL,NYJ,1.0,1.0,10.0,60.0,0.0,0.488917,5.883399,3.732620,outside run
8,2022,1,2022_01_BAL_NYJ,BAL,NYJ,1.0,2.0,10.0,60.0,0.0,0.488917,5.883399,3.732620,short pass left
9,2022,1,2022_01_BAL_NYJ,BAL,NYJ,1.0,3.0,2.0,52.0,0.0,0.488917,5.883399,3.732620,outside run


In [30]:
df.isnull().sum()

season                     0
week                       0
game_id                    0
posteam                    0
defteam                    0
qtr                        0
down                       0
ydstogo                    0
yardline_100               0
score_differential         0
def_pass_rate_last3       62
avg_yds_pp_alwd_last3     62
avg_rush_pp_alwd_last3    62
play_subtype               0
dtype: int64

In [31]:
df.shape

(65115, 14)

There are going to be some null values in these historical aggregate columns. This is due to the fact that this model considers prior games up to the start of the season. So there are not always enough games to have a historical average. Because of this, we will drop null values for simplicity and ease of modeling. 

In [33]:
df = df.dropna()

In [34]:
df.isnull().sum()

season                    0
week                      0
game_id                   0
posteam                   0
defteam                   0
qtr                       0
down                      0
ydstogo                   0
yardline_100              0
score_differential        0
def_pass_rate_last3       0
avg_yds_pp_alwd_last3     0
avg_rush_pp_alwd_last3    0
play_subtype              0
dtype: int64

In [35]:
df.shape

(65053, 14)

In [36]:
df.head()

,season,week,game_id,posteam,defteam,qtr,down,ydstogo,yardline_100,score_differential,def_pass_rate_last3,avg_yds_pp_alwd_last3,avg_rush_pp_alwd_last3,play_subtype
0,2022,1,2022_01_BAL_NYJ,NYJ,BAL,1.0,1.0,10.0,78.0,0.0,0.656250,5.358173,4.320055,outside run
1,2022,1,2022_01_BAL_NYJ,NYJ,BAL,1.0,1.0,10.0,59.0,0.0,0.656250,5.358173,4.320055,short pass left
2,2022,1,2022_01_BAL_NYJ,NYJ,BAL,1.0,2.0,10.0,59.0,0.0,0.656250,5.358173,4.320055,outside run
3,2022,1,2022_01_BAL_NYJ,NYJ,BAL,1.0,3.0,5.0,54.0,0.0,0.656250,5.358173,4.320055,short pass right
4,2022,1,2022_01_BAL_NYJ,BAL,NYJ,1.0,1.0,10.0,72.0,0.0,0.488917,5.883399,3.732620,short pass right


This is where I want the data to be after cleaning and preprocessing. Next steps will be to explore the modeling process of this project (see 02_modeling.ipynb) to ensure that no further preprocessing needs to be done. Eventually the steps taken in this notebook to preprocess data will be ported to C++ for efficiency. 

In [70]:
df.to_csv('cleaned_pbp_22_to_24.csv', index=False)